This notebook produces defining relations for $G_K/D_4(G_K)$ for $K=\mathbb{Q}(\sqrt{d})$, for every $d$ in our list of discriminants. The result is a list of defining relations for groups, which can then by plugged into GAP to compute the small group ids of these groups.       

In [1]:
def check_p_rank (K_cyc, p):
    nr = 0
    for l in K_cyc:
        c = int(l)
        if c%p == 0:
            nr += 1
    return nr

print("The following line should read [3, 2, 2, 3]")
[check_p_rank(['3', '27', '3'], 3),
    check_p_rank(['3', '45', '5'], 3),
    check_p_rank(['3', '27'], 3),
    check_p_rank(['5', '25', '5'], 5)]

The following line should read [3, 2, 2, 3]


[3, 2, 2, 3]

In [3]:
def check_p_sq_rank (K_cyc, p):
    nr = 0
    for l in K_cyc:
        c = int(l)
        if c%(p**2) == 0:
            nr += 1
    return nr

print("The following line should read [1, 1, 2, 1]")
[check_p_sq_rank(['3', '27', '3'], 3),
    check_p_sq_rank(['3', '45', '5'], 3),
    check_p_sq_rank(['9', '27'], 3),
    check_p_sq_rank(['5', '25', '5'], 5)]

The following line should read [1, 1, 2, 1]


[1, 1, 2, 1]

In [ ]:
import re, subprocess

#executable_path = '/Users/eric/Documents/Matematik/cup-products/main-pol'
executable_path = '../Massey/build/massey'
p = "3"
pat = re.compile(r'(?<=\^)[+-]?\d+(?!,)(?=[A-Za-z(])')

def run_once(command, timeout=10):
    try:
        r = subprocess.run(command, capture_output=True, text=True,
                           timeout=timeout, check=False, close_fds=True)
        return r.stdout, r.stderr, r.returncode
    except subprocess.TimeoutExpired:
        return None, None, 'TIMEOUT'

infile  = "discriminants_3_cyc_[3^m,3^n]"
outfile = "gap-files/all_rels"

first = True
with open(outfile, "a") as out:
    out.write("rels:=[")
    with open(infile, "r") as f:
        for i, line in enumerate(f):
            # if i < 0:
            #     continue
            D = line.strip()[1:]
            pol = "s^2+" + D
            print(f"{i} : D: {D}", flush=True)

            stdout, stderr, rc = run_once([executable_path, p, pol], timeout=300)
            if rc == 'TIMEOUT' or stdout is None:
                print(f"Skipped (timeout/runner issue): D={D}")
                continue

            full = stdout
            a = full.find('""""'); b = full.find('""""', a + 4)
            c = full.find('"""');  d = full.find('"""',  c + 3)

            if c == -1 or d == -1 or a == -1 or b == -1:
                print(f"Missing markers, skipping D={D}")
                continue

            output_c = full[c+3:d]
            output_m = full[a+4:b]
            modified_m = pat.sub(r'\g<0>*', output_m)

            if not first:
                out.write(",\n")
            out.write(f"[{D}, {output_c}, {modified_m}]")
            first = False

    out.write("];\n")